In [1]:
from moviepy.editor import *
import os
from concurrent.futures import ProcessPoolExecutor

In [18]:

def create_video_from_folder(folder, gif_file=None):
    print("Start creating from: 📂", folder)

    # Путь к аудио-файлу
    audio_file = os.path.join(folder, "music.mp3")

    # Проверяем, существует ли аудио-файл
    if not os.path.isfile(audio_file):
        print(f"❌Audio file not found in {folder}")
        return

    # Список изображений в папке
    images = [os.path.join(folder, img) for img in sorted(os.listdir(folder)) if img.endswith(('.png', '.jpg', '.jpeg'))]

    # Длительность аудио-файла
    audio = AudioFileClip(audio_file)
    audio_duration = audio.duration

    # Длительность каждого изображения (в секундах)
    image_duration = 2  # Измените на желаемую длительность

    # Создаем список клипов из изображений
    image_clips = []
    for img in images:
        clip = ImageClip(img).set_duration(image_duration)
        print("Load image: 🖼", img)
        image_clips.append(clip)

    # Если нет изображений, используем черный фон
    if not image_clips:
        image_clips = [ColorClip(size=(1280, 720), color=(0, 0, 0)).set_duration(image_duration)]

    # Объединяем клипы в одно слайд-шоу
    slideshow = concatenate_videoclips(image_clips, method="compose")

    # Зацикливаем слайд-шоу на всю длительность аудио
    slideshow = slideshow.loop(duration=audio_duration)

    # Проверяем наличие GIF-файла и накладываем его на видео
    if gif_file and os.path.isfile(gif_file):
        print("✔Gif file found")
        # Загружаем GIF и зацикливаем на всю длительность аудио
        gif_clip = (VideoFileClip(gif_file, has_mask=True)
                    .loop(duration=audio_duration)
                    # .resize(0.5)  # Масштабирование (0.5 = 50% от исходного размера)
                    .set_position(("left", "bottom")))  # Позиция (можно изменить на нужную)

        # Делаем фон GIF прозрачным (удаляем определенный цвет)
        gif_clip = gif_clip.fx(vfx.mask_color, color=[0, 0, 0], thr=100, s=5)

        # Накладываем GIF поверх слайд-шоу
        final_video = CompositeVideoClip([slideshow, gif_clip])
    else:
        final_video = slideshow

    # Добавляем аудио к видео
    final_video = final_video.set_audio(audio)

    # Сохраняем финальное видео
    output_file = f"{folder}_output_video.mp4"
    final_video.write_videofile(output_file, fps=24)
    print(f"Video created: {output_file}")


In [19]:

def process_folders(base_folder, num_workers=1):
    # Поиск всех папок, начинающихся с "Clip"
    folders = [os.path.join(base_folder, folder) \
               for folder in sorted(os.listdir(base_folder)) \
               if folder.startswith("Clip") and len(folder) == 5]

    print('Folders found: ', folders)

    # Путь к GIF-файлу (если есть)
    gif_file = base_folder + "static/animated2.gif"  # Вы можете указать путь к вашему GIF-файлу

    # Используем параллельную обработку
    # with ProcessPoolExecutor(max_workers=num_workers) as executor:
    #     executor.map(create_video_from_folder, folders, [gif_file]*len(folders))
    create_video_from_folder(folders[0], gif_file)


In [20]:

# Основной запуск
# if __name__ == "__main__":

base_folder = "../"  # Укажите путь к основной папке, содержащей папки Clip
num_workers = 1  # Количество параллельных процессов

process_folders(base_folder, num_workers)


Folders found:  ['../Clip1', '../Clip1_output_video.mp4']
Start creating from: 📂 ../Clip1
Load image: 🖼 ../Clip1\image1.png
Load image: 🖼 ../Clip1\image2.png
Load image: 🖼 ../Clip1\image3.png
Load image: 🖼 ../Clip1\image4.png
Load image: 🖼 ../Clip1\image5.png
✔Gif file found
Moviepy - Building video ../Clip1_output_video.mp4.
MoviePy - Writing audio in Clip1_output_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ../Clip1_output_video.mp4



t:  49%|████▉     | 114/232 [02:28<02:46,  1.41s/it, now=None]